# **HOMEWORK 1 - Regressione Lineare**

In questo homework dovrete:

1. Scrivere una funzione di pipeline che deve gestire l' allenamento di un modello di regressione lineare al variare degli iperparametri forniti. Nello specifico:
    * Deve applicare la PCA, se presente.
    
    * Deve applicare la standardizzazione, se presente.

    * Deve applicare la regolarizzazione, se presente.

    * Deve allenare il modello di regressione lineare.

    * Deve calcolare la MAE.

2. Scrivere una funzione che utilizzi la `pipeline` definita al punto 1 e che testi tutte le configurazioni possibili presenti in `configs`. Nel dettaglio la funzione deve:
    * Dividere il dataset in train e validation.

    * Calcolare, grazie alla funzione `pipeline` definita al punto 1, quale configurazione ottiene il punteggio migliore (quale configurazione ha la MAE di validation più bassa).

3. Scrivere una funzione che utilizzi la configurazione migliore prodotta dalla funzione definita al punto 2 e la testi sul test set. 

4. Stampare:
    * La migliore configurazione

    * Il miglior MAE di validation 

    * Il migliore MAE di train

    * Il MAE di test 


Il codice che di seguito trovate già fornito deve essere utilizzato per la risoluzione dell' homework, **NON MODIFICATELO IN ALCUN MODO**.

## **Dataset Wine Quality White**

Il dataset da utilizzare è `wine-quality-white` della libreria `scikit-learn`. Il dataset contiene 11 variabili numeriche + 1 di target che classifica il vino in diverse categorie di qualità. Per il nostro obiettivo la variabile di target è considerata come `float`, permettendoci di applicare la regressione lineare. All' interno del dataset sono contenuti 4898 campioni. 

In [1]:
# Questa cella contiene tutte le librerie di cui necessitate per risolvere l' homework.
# Ricordate di eseguirla prima di iniziare.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_openml
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler

In [2]:
hyperparams = {
    # PCA
    'use_pca': [True, False],
    'pca_standardize': [True, False],
    'pca_components': [3, 5, 10],
    # Data standardization
    'data_standardize': [True, False],
    # Regularization l2
    'use_regularization': [True, False],
    'reg_lambda': [0.1, 1, 10],
}

# Calcoliamo tutte le possibili combinazioni di iperparametri
import itertools
combinations = list(itertools.product(*hyperparams.values()))
configs = [dict(zip(hyperparams.keys(), combination)) for combination in combinations]

# Evitiamo le combinazioni non valide
for config in configs:
    if not config['use_pca']:
        config['pca_standardize'] = None
        config['pca_components'] = None
    if not config['use_regularization']:
        config['reg_lambda'] = None
configs = set([tuple(config.items()) for config in configs])

# Convertiamo di nuovo in lista di dizionari
configs = [dict(config) for config in configs]
print(f'Numero di combinazioni: {len(configs)}')

Numero di combinazioni: 56


In `configs` avete una lista di dizionari, ogni dizionario contiene una possibile combinazione di hyperparametri da utilizzare nella fase di training. 

In [3]:
# Carica il dataset Wine Quality White
data = fetch_openml(name='wine-quality-white', version=1, as_frame=True)
X = data.data
y = data.target.astype(float)  # Assicura che il target sia float per la regressione

def pipeline(X_train, y_train, X_val, y_val, hyperparams):
    """
    Addestra un modello di regressione lineare con eventuale PCA e regolarizzazione L2.
    """

    # Copia dei dati per evitare modifiche sui dati originali
    X_train = X_train.copy()
    X_val = X_val.copy()

    if hyperparams['use_pca']:
        n_components = hyperparams['pca_components']
        pca = PCA(n_components=n_components)
        X_train = pca.fit_transform(X_train)
        X_val = pca.transform(X_val)

        if hyperparams['pca_standardize']:
            #Standardizzazione dati dopo PCA
            scaler = StandardScaler()
            X_train = scaler.fit_transform(X_train)
            X_val = scaler.transform(X_val)

    if hyperparams['data_standardize']:
        #Standardizzazione dati se non si usa PCA
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)

    # Aggiunge il termine costante ai dati
    X_train = np.c_[np.ones((X_train.shape[0], 1)), X_train] 
    X_val = np.c_[np.ones((X_val.shape[0], 1)), X_val]

    # Calcolo della soluzione di regressione lineare
    if hyperparams['use_regularization']:
        # Regolarizzazione L2: utilizziamo Ridge regression
        from sklearn.linear_model import Ridge
        model = Ridge(alpha=hyperparams['reg_lambda'])
    else:
        # Senza regolarizzazione: regressione lineare standard
        from sklearn.linear_model import LinearRegression
        model = LinearRegression()

    # Addestriamo il modello
    model.fit(X_train, y_train)

    # Calcolo predizioni
    y_train_pred = model.predict(X_train)
    y_val_pred = model.predict(X_val)
    
    # Calcola il MAE
    MAE_train = np.mean(np.abs(y_train - y_train_pred))
    MAE_val = np.mean(np.abs(y_val - y_val_pred))

    return MAE_train, MAE_val

# Dividi il dataset in training e test set
train_fraction = 0.6  # 60% --> training
test_fraction = 0.2   # 20% --> test
validation_fraction = 0.20  # 20% --> validation
    #Divisione dataset training set e test set
num_train = int(train_fraction * X.shape[0])
X_train = X[:num_train]
y_train = y[:num_train]
X_test = X[num_train:]
y_test = y[num_train:]


# Dividi il training set in training set effettivo e validation set
num_val = int(validation_fraction * X_train.shape[0])

# Creazione del validation set
X_val = X_train[-num_val:]
y_val = y_train[-num_val:]

# Aggiorna X_train e y_train, escludendo la parte di validation set
X_train = X_train[:-num_val]
y_train = y_train[:-num_val]

# Trova la configurazione di iperparametri migliore
best_config = None
best_mae_val = float('inf')
best_mae_train = None

for config in configs:
    MAE_train, MAE_val = pipeline(X_train, y_train, X_val, y_val, config)
    
    if MAE_val < best_mae_val:
        best_mae_val = MAE_val
        best_mae_train = MAE_train
        best_config = config

# Riallena il modello sul training set completo
X_full_train = np.vstack((X_train, X_val))
y_full_train = np.concatenate((y_train, y_val))

# Calcola il MAE sul test set
mae_train_final, mae_val_final = pipeline(X_full_train, y_full_train, X_test, y_test, best_config)

# Stampa  risultati 
print(f"Migliore configurazione: {best_config}")
print(f"Miglior MAE di validation: {best_mae_val}")
print(f"Miglior MAE di train: {best_mae_train}")
print(f"MAE finale test set: {mae_val_final}")

/usr/local/lib/python3.11/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


Migliore configurazione: {'use_pca': False, 'pca_standardize': None, 'pca_components': None, 'data_standardize': False, 'use_regularization': True, 'reg_lambda': 1}
Miglior MAE di validation: 0.5758382045360769
Miglior MAE di train: 0.6110774467088591
MAE finale test set: 0.5776071103930652
